<a href="https://colab.research.google.com/github/nahyunryou/ML_Project/blob/main/Catboost4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import warnings
warnings.filterwarnings('ignore')

import glob
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
import random

import optuna
from optuna.samplers import TPESampler

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
train = pd.read_csv('/content/drive/MyDrive/ml_project/processed_train_code.csv')
#train = train.drop(['index'], axis=1)
train.fillna('NAN',inplace=True)

test = pd.read_csv('/content/drive/MyDrive/ml_project/processed_test_code.csv')
#test = test.drop(['index'],axis=1)
test.fillna('NAN',inplace=True)

submit = pd.read_csv('/content/drive/MyDrive/ml_project/sample_submission.csv')

In [22]:
edu_dict = {'Lower secondary':1,
 'Secondary / secondary special':2,
 'Incomplete higher':3,
 'Higher education':4,
 'Academic degree':5}

train['edu_type'].replace(edu_dict,inplace=True)
test['edu_type'].replace(edu_dict,inplace=True)

In [23]:
house_dict = {'With parents':3,
 'Co-op apartment':2,
 'Municipal apartment':1,
 'Rented apartment':4,
 'Office apartment':5,
 'House / apartment':6}

train['house_type'].replace(house_dict,inplace=True)
test['house_type'].replace(house_dict,inplace=True)

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24817 entries, 0 to 24816
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         24817 non-null  int64  
 1   gender             24817 non-null  object 
 2   car                24817 non-null  object 
 3   reality            24817 non-null  object 
 4   income_total       24817 non-null  float64
 5   income_type        24817 non-null  object 
 6   edu_type           24817 non-null  int64  
 7   family_type        24817 non-null  object 
 8   house_type         24817 non-null  int64  
 9   work_phone         24817 non-null  int64  
 10  home_phone         24817 non-null  int64  
 11  email              24817 non-null  int64  
 12  occup_type         24817 non-null  object 
 13  family_size        24817 non-null  float64
 14  begin_month        24817 non-null  int64  
 15  credit             24817 non-null  float64
 16  CODE               248

#Training

In [25]:
#Optuna용 Train셋
X = train.drop(['credit'],axis=1)
y = train['credit']
X_test = test.copy()

In [56]:
def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }

  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  
  
  cat_features =[0,1,2,3,5,6,7,8,12,15]
  #cat_features =[1,2,3,5,7,12,16]
  cat = CatBoostClassifier(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict_proba(X_valid)
  log_score = log_loss(y_valid, cat_pred)

  return log_score

In [57]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-12-08 15:18:10,546] A new study created in memory with name: cat_parameter_opt


0:	learn: 1.0831813	test: 1.0831813	test1: 1.0834314	best: 1.0834314 (0)	total: 15.8ms	remaining: 1m 59s
100:	learn: 0.8863654	test: 0.8863654	test1: 0.8910306	best: 0.8910306 (100)	total: 1.25s	remaining: 1m 32s


[I 2022-12-08 15:18:13,121] Trial 0 finished with value: 0.8909113844295762 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 7588, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.8909113844295762.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.8909113844
bestIteration = 151

Shrink model to first 152 iterations.
0:	learn: 0.9953120	test: 0.9953120	test1: 0.9958664	best: 0.9958664 (0)	total: 94.7ms	remaining: 4m 35s
100:	learn: 0.7032696	test: 0.7143863	test1: 0.8012717	best: 0.8009552 (97)	total: 8.84s	remaining: 4m 5s
200:	learn: 0.6123847	test: 0.6331863	test1: 0.7911360	best: 0.7904294 (195)	total: 17.6s	remaining: 3m 56s


[I 2022-12-08 15:18:34,681] Trial 1 finished with value: 0.7891585836657066 and parameters: {'learning_rate': 0.2708160864249968, 'bagging_temperature': 21.368329072358772, 'n_estimators': 2911, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7891585836657066.


...

0:	learn: 1.0179462	test: 1.0181537	test1: 1.0180632	best: 1.0180632 (0)	total: 161ms	remaining: 5m 33s
100:	learn: 0.2692545	test: 0.4400574	test1: 0.7737339	best: 0.7145968 (69)	total: 1m 4s	remaining: 20m 52s


[I 2022-12-08 16:06:39,184] Trial 8 finished with value: 0.7145967847220132 and parameters: {'learning_rate': 0.20441878352493792, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 2076, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.6974005861729224.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7145967847
bestIteration = 69

Shrink model to first 70 iterations.
0:	learn: 1.0923012	test: 1.0923097	test1: 1.0924210	best: 1.0924210 (0)	total: 94.8ms	remaining: 10m 37s
100:	learn: 0.8751675	test: 0.8424497	test1: 0.8748033	best: 0.8748033 (100)	total: 12.1s	remaining: 13m 13s
200:	learn: 0.8466512	test: 0.7749177	test1: 0.8410227	best: 0.8410227 (200)	total: 24.9s	remaining: 13m 27s
300:	learn: 0.8305746	test: 0.7313296	test1: 0.8204521	best: 0.8204490 (299)	total: 32.7s	remaining: 11m 38s
400:	learn: 0.8168867	test: 0.6867078	test1: 0.8025546	best: 0.8025546 (400)	total: 42.8s	remaining: 11m 14s
500:	learn: 0.8079207	test: 0.6562408	test1: 0.7911614	best: 0.7911614 (500)	total: 52.3s	remaining: 10m 50s
600:	learn: 0.7996172	test: 0.6390059	test1: 0.7820648	best: 0.7820648 (599)	total: 1m	remaining: 10m 15s
700:	learn: 0.7902007	test: 0.6170433	test1: 0.7709201	best: 0.7709200 (696)	total: 1m 7s	remaining: 9m 40s

[I 2022-12-08 16:09:43,080] Trial 9 finished with value: 0.7068321239017791 and parameters: {'learning_rate': 0.01443340240633889, 'bagging_temperature': 0.0133572404119741, 'n_estimators': 6728, 'max_depth': 8, 'random_strength': 51, 'colsample_bylevel': 0.9445398843556558, 'l2_leaf_reg': 7.486273952174759e-06, 'min_child_samples': 44, 'max_bin': 427, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.6974005861729224.


Best Score: 0.6974005861729224
Best trial {'learning_rate': 0.027010527749605478, 'bagging_temperature': 0.2920433847181412, 'n_estimators': 5105, 'max_depth': 14, 'random_strength': 20, 'colsample_bylevel': 0.708540663048167, 'l2_leaf_reg': 1.7776512920172654e-05, 'min_child_samples': 9, 'max_bin': 382, 'od_type': 'IncToDec'}


In [58]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['credit']):
  folds.append((train_idx,valid_idx))

In [59]:
random.seed(42)
cat_models={}

cat_features =[0,1,2,3,5,6,7,8,12,15]

for fold in range(5):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['credit'],axis=1).iloc[train_idx].values 
  X_valid = train.drop(['credit'],axis=1).iloc[valid_idx].values
  y_train = train['credit'][train_idx].values
  y_valid = train['credit'][valid_idx].values

  cat = CatBoostClassifier(**study.best_params)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

====================================1============================================
0:	learn: 1.0869740	test: 1.0869740	test1: 1.0869753	best: 1.0869753 (0)	total: 23.6ms	remaining: 2m
100:	learn: 0.8166111	test: 0.7245872	test1: 0.8003858	best: 0.8003858 (100)	total: 40.5s	remaining: 33m 44s
200:	learn: 0.7770823	test: 0.6356109	test1: 0.7545789	best: 0.7545789 (200)	total: 1m 23s	remaining: 34m 18s
300:	learn: 0.7606102	test: 0.6009085	test1: 0.7384179	best: 0.7384179 (300)	total: 1m 38s	remaining: 26m 21s
400:	learn: 0.7416235	test: 0.5639451	test1: 0.7216948	best: 0.7216948 (400)	total: 2m 14s	remaining: 26m 16s
500:	learn: 0.6087536	test: 0.5058471	test1: 0.6994997	best: 0.6993562 (499)	total: 4m 27s	remaining: 41m 1s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6988383494
bestIteration = 519

Shrink model to first 520 iterations.


====================================2============================================
0:	learn: 1.0869740	test: 1.0869740	test1: 1.08

#Test

In [60]:
submit.iloc[:,1:]=0
for fold in range(5):
  submit.iloc[:,1:] += cat_models[fold].predict_proba(test)/5

In [61]:
import datetime
now = datetime.datetime.now()
created_time = now.strftime('%m%d-%H%M')

submit.to_csv(f'/content/drive/MyDrive/ml_project/{created_time}_submit.csv',index=False)

In [62]:
submit.head(20)

,index,0,1,2
0,26457,0.116563,0.184823,0.698614
1,26458,0.256895,0.275516,0.467589
2,26459,0.043685,0.089197,0.867118
3,26460,0.054692,0.096069,0.849240
4,26461,0.091750,0.262117,0.646133
5,26462,0.065920,0.189332,0.744748
6,26463,0.432308,0.523684,0.044008
7,26464,0.055282,0.122433,0.822286
8,26465,0.063988,0.109857,0.826155
9,26466,0.049588,0.302470,0.647942
